<a href="https://colab.research.google.com/github/nadireus/Clarusway/blob/main/SQL_and_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import sqlite3

In [29]:
data= sqlite3.connect('example.db')

In [36]:
sql = ''' CREATE TABLE transactions (
    Sender_ID int,
    Receiver_ID int,
    Amount int,
    Transaction_Date DATE
    
);



INSERT INTO transactions (Sender_ID, Receiver_ID, Amount, Transaction_Date)
VALUES 
('55','22', '500','18.05.2021'),
('11','33', '350','19.05.2021'),
('22','11', '650','19.05.2021'),
('22','33', '900','20.05.2021'),
('33','11', '500','21.05.2021'),
('33','22', '750','21.05.2021'),
('11','44', '300','22.05.2021');
 '''

In [37]:
cur=data.cursor()

In [38]:
cur.executescript(sql)

In [39]:
sql = '''select * from transactions '''

In [40]:
pd.read_sql(sql, data)

,Sender_ID,Receiver_ID,Amount,Transaction_Date
0,55,22,500,18.05.2021
1,11,33,350,19.05.2021
2,22,11,650,19.05.2021
3,22,33,900,20.05.2021
4,33,11,500,21.05.2021
5,33,22,750,21.05.2021
6,11,44,300,22.05.2021


In [48]:
sql = '''SELECT	*
FROM	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
FULL OUTER JOIN	
		(
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
ON		A.Sender_Id = B.Receiver_Id;
'''

In [49]:
pd.read_sql(sql, data) # FULL OUTER JOIN gives error becouse RIGHT and FULL OUTER JOINs are not currently supported
                       # so we need to use LEFT JOIN as below

DatabaseError: ignored

In [50]:
sql = '''SELECT	*
FROM	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
LEFT JOIN	
		(
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
ON		A.Sender_Id = B.Receiver_Id

UNION ALL
SELECT	*
FROM  (
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
LEFT JOIN	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
ON		A.Sender_Id = B.Receiver_Id;
'''



In [51]:
pd.read_sql(sql, data)

,Sender_Id,s_amount,Receiver_Id,r_amount
0,11,650,11.0,1150.0
1,22,1550,22.0,1250.0
2,33,1250,33.0,1250.0
3,55,500,NaN,NaN
4,11,1150,11.0,650.0
5,22,1250,22.0,1550.0
6,33,1250,33.0,1250.0
7,44,300,NaN,NaN


In [66]:
sql = '''SELECT	COALESCE(A.sender_id, B.receiver_id) AS Account_ID,
		            COALESCE(B.r_amount, 0) - COALESCE(A.s_amount, 0) AS Net_Change
FROM	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
LEFT JOIN	
		(
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
ON		A.Sender_Id = B.Receiver_Id

UNION ALL
SELECT	COALESCE(A.sender_id, B.receiver_id) AS Account_ID,
		    COALESCE(B.r_amount, 0) - COALESCE(A.s_amount, 0) AS Net_Change
FROM  (
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
LEFT JOIN	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
ON		A.Sender_Id = B.Receiver_Id
WHERE  A.Sender_Id  IS NULL
;
'''

In [67]:
pd.read_sql(sql, data)

,Account_ID,Net_Change
0,11,500
1,22,-300
2,33,0
3,55,-500
4,44,300


In [68]:
sql = '''SELECT	COALESCE(A.sender_id, B.receiver_id) AS Account_ID,
		            COALESCE(B.r_amount, 0) - COALESCE(A.s_amount, 0) AS Net_Change
FROM	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
LEFT JOIN	
		(
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
ON		A.Sender_Id = B.Receiver_Id

UNION ALL
SELECT	COALESCE(A.sender_id, B.receiver_id) AS Account_ID,
		    COALESCE(B.r_amount, 0) - COALESCE(A.s_amount, 0) AS Net_Change
FROM  (
		SELECT	Receiver_Id, SUM(amount) AS r_amount
		FROM		transactions
		GROUP BY Receiver_Id
		) B
LEFT JOIN	(
		SELECT	Sender_Id, SUM(amount) AS s_amount
		FROM	transactions
		GROUP BY Sender_Id
		) A
ON		A.Sender_Id = B.Receiver_Id
WHERE  A.Sender_Id  IS  NULL
ORDER BY Net_Change DESC
;
'''

In [69]:
pd.read_sql(sql, data)

,Account_ID,Net_Change
0,11,500
1,44,300
2,33,0
3,22,-300
4,55,-500
